In [19]:
import pandas as pd
import datetime as dt

# Objectif du notebook

A partir des prédictions réalisées sur la Consommation et les productions éolienne, solaire et hydraulique, l'objectif est de "calculer" la balance en considérant :
* la production bioénergie est négligeable
* la production thermique est nulle (arrêt de ces installations)
* la production nucléaire est une constante qui correspond à un pourcentage de la capacité disponible (taux de charge).

NB : le taux de charge permet de prendre en compte les phases de maintenance des installations concernées.

In [15]:
solaire = pd.read_csv('./results/bilan_solaire_3h.csv', sep = ';',
                     usecols = ['Dates', 'Sol_p_exog'])
hydraulique = pd.read_csv('./results/bilan_hydraulique_3h.csv', sep = ';',
                     usecols = ['Hyd_p_exog'])
eolien = pd.read_csv('./results/bilan_eolien_3h.csv', sep = ';',
                     usecols = ['Eol_p_LinReg'])
consommation = pd.read_csv('./results/bilan_consommation_3h.csv', sep = ';',
                     usecols = ['Conso_p_exog'])

In [12]:
df = pd.concat([solaire, hydraulique, eolien, consommation], axis = 1)
df

,Dates,Sol_p_exog,Hyd_p_exog,Eol_p_LinReg,Conso_p_exog
0,2020-01-01 00:00:00,255.256338,6023.128000,3027.765656,57857.919526
1,2020-01-01 03:00:00,367.021775,4979.754542,3178.135821,51303.514166
2,2020-01-01 06:00:00,286.951381,5543.163473,3476.931639,48891.957506
3,2020-01-01 09:00:00,865.248916,5982.979775,3700.630327,51152.626500
4,2020-01-01 12:00:00,1543.529684,6289.655194,4677.235794,54619.650062
...,...,...,...,...,...
5355,2021-10-31 09:00:00,6339.724663,3355.074004,8130.668099,48842.505423
5356,2021-10-31 12:00:00,2308.136636,2958.404035,9365.458055,47385.394859
5357,2021-10-31 15:00:00,-158.710386,5922.379878,8973.176619,39895.231968
5358,2021-10-31 18:00:00,159.941565,3101.525246,8629.518193,47866.660937


In [16]:
df['Balance'] = df['Sol_p_exog'] + df['Eol_p_LinReg'] + df['Hyd_p_exog'] - df['Conso_p_exog']
df['Dates'] = pd.to_datetime(df['Dates'], yearfirst = True)
df.set_index('Dates', inplace = True)
df

,Sol_p_exog,Hyd_p_exog,Eol_p_LinReg,Conso_p_exog,Balance
Dates,,,,,
2020-01-01 00:00:00,255.256338,6023.128000,3027.765656,57857.919526,-48551.769534
2020-01-01 03:00:00,367.021775,4979.754542,3178.135821,51303.514166,-42778.602028
2020-01-01 06:00:00,286.951381,5543.163473,3476.931639,48891.957506,-39584.911014
2020-01-01 09:00:00,865.248916,5982.979775,3700.630327,51152.626500,-40603.767482
2020-01-01 12:00:00,1543.529684,6289.655194,4677.235794,54619.650062,-42109.229391
...,...,...,...,...,...
2021-10-31 09:00:00,6339.724663,3355.074004,8130.668099,48842.505423,-31017.038658
2021-10-31 12:00:00,2308.136636,2958.404035,9365.458055,47385.394859,-32753.396134
2021-10-31 15:00:00,-158.710386,5922.379878,8973.176619,39895.231968,-25158.385857


In [22]:
balance = pd.read_csv('./dataset/France_3h.csv',
                     usecols = ['Dates', 'Balance'])
balance['Dates'] = pd.to_datetime(balance['Dates'], yearfirst = True)
balance.set_index('Dates', inplace = True)
balance = balance[(balance.index >= dt.datetime(2020,1,1))&(balance.index < dt.datetime(2021,11,1))]
balance

,Balance
Dates,
2020-01-01 00:00:00,4203.833333
2020-01-01 03:00:00,3650.666667
2020-01-01 06:00:00,2513.000000
2020-01-01 09:00:00,1314.166667
2020-01-01 12:00:00,695.166667
...,...
2021-10-31 09:00:00,7665.666667
2021-10-31 12:00:00,7709.166667
2021-10-31 15:00:00,8713.166667
